<a href="https://colab.research.google.com/github/sanaaria/Master-thesis/blob/main/KGE_NFM_%26_NFM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
################################
#This script provide a demo of KGE_NFM & NFM, the runtime on one fold mainly takes 40~50 minutes.


In [13]:
#  !pip uninstall ampligraph

In [14]:
# pip install rdflib==7.0.0

In [15]:
# pip install scipy==1.10.0

In [3]:
pip install tensorflow==2.13

In [4]:
import tensorflow as tf
print(tf.__version__)

2.13.0


In [5]:
pip install scikit-learn==0.24.2

  Using cached scikit-learn-0.24.2.tar.gz (7.5 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (pyproject.toml) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [6]:
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder,MinMaxScaler

In [7]:
pip install deepctr

In [8]:
pip install ampligraph

In [9]:
pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: /usr/local/lib/python3.10/dist-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: ampligraph, bigframes, fastai, imbalanced-learn, librosa, mlxtend, qudida, sklearn-pandas, yellowbrick


In [10]:
import ampligraph as ampligraph

In [11]:
print(ampligraph.__version__)

2.0.1


In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import pandas as pd
import ampligraph as ampligraph
from ampligraph.datasets import load_from_csv

In [13]:
# from ampligraph.evaluation import train_test_split_no_unseen,generate_corruptions_for_fit
# # from ampligraph.evaluation import train_test_split_no_unseen
from ampligraph.datasets import load_from_csv


In [14]:
# from ampligraph.evaluation import evaluate_performance
# As of version 1.1.1, Ampligraph removed the 'evaluate_performance' function and instead introduced the 'mrr_score', 'hits_at_k', and 'mean_rank' functions to calculate metrics for evaluating model performance.
# If you are using version 2.0.1, you should be able to use the 'mrr_score', 'hits_at_k', and 'mean_rank' functions to calculate the desired metrics. Here's an example of how you can do this
from ampligraph.evaluation import mrr_score

In [15]:
from ampligraph.evaluation import mrr_score, hits_at_n_score ,mr_score

In [16]:
# from ampligraph.evaluation.common import generate_corruptions
from ampligraph.latent_features.layers.corruption_generation import CorruptionGenerationLayerTrain

In [17]:
# from ampligraph import ComplEx,TransE,DistMult

In [18]:
from ampligraph.latent_features.layers.scoring import ComplEx

In [19]:
from ampligraph.latent_features.layers.scoring import TransE

In [20]:
from ampligraph.latent_features.layers.scoring import DistMult

In [21]:
# from ampligraph.evaluation import evaluate_performance

In [22]:
# from ampligraph.utils import save_model,restore_model
from ampligraph.utils import save_model
from ampligraph.utils import restore_model

In [23]:
import deepctr

In [24]:
from tensorflow.python.keras.models import Model

In [25]:
from tensorflow.python.keras.layers import Dense, Dropout

In [26]:
print(deepctr.__version__)

0.9.3


In [27]:
from keras.layers import LSTM, Lambda, Layer, Dropout

In [28]:
# from tensorflow.python.keras.layers import LSTM, Lambda, Layer, Dropout

In [29]:
# import tf.keras.layers.LSTM

In [30]:
# NFM stands for Neural Factorization Machine, which is a type of neural network designed for recommendation systems.
from deepctr.models import NFM

In [31]:
from deepctr.feature_column import SparseFeat,DenseFeat,get_feature_names

In [32]:
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam,Adagrad,Adamax


In [33]:
from tensorflow import keras

In [34]:



#load data
################################################################

from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
#data example: yamanishi_08
dt_08 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt',delimiter='\t',header=None)
# the script reads a csv file using pandas' read_csv function. This function reads the file from the specified path, which in this case is
# /content/drive/MyDrive/data/yamanishi_08/dt_all_08.txt.

dt_08.columns = ['head','relation','tail']
# the columns of the DataFrame dt_08 are set using the columns attribute. The column names are 'head', 'relation', and 'tail'.

In [36]:
#kg
# ##This code is written in Python using the pandas library.
# #The goal of this code is to load two text files,
# which contain Knowledge Graph (KG) data, and concatenate them into a single pandas DataFrame.
# The KG data in these text files consists of triples (head, relation, tail), which are essentially edges in a graph.
# The 'head' is the subject, the 'relation' is the predicate, and the 'tail' is the object.

kg1 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/kg_data/kegg_kg.txt',delimiter='\t',header=None)
# The pd.read_csv() function reads the specified file and creates a DataFrame. The delimiter='\t' argument tells pandas to use tabs as separators.
# The header=None argument tells pandas that the first row of the file does not contain column names.

kg2 = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/kg_data/yamanishi_uniprot_kg.txt',delimiter='\t',header=None)
#This code is similar to the previous one.

kg = pd.concat([kg1,kg2])
#Concatenate the two DataFrames.
#The pd.concat() function concatenates the input DataFrames into a single DataFrame.

kg.index = range(len(kg))
#Reset the index of the concatenated DataFrame.
#The index attribute of a DataFrame represents the index of the rows.
#This line of code resets the index of the concatenated DataFrame so that it starts from 0 and increments by 1.

kg.columns = ['head','relation','tail']
#Set the column names of the concatenated DataFrame.
#This line of code assigns new column names to the concatenated DataFrame.


#The resulting kg DataFrame contains the combined KG data from both text files.
# The DataFrame has three columns: 'head', 'relation', and 'tail'. The rows represent the triples (head, relation, tail) in the KG.

In [37]:
##This code is a preprocessing step for the neural feature matrix (NFM) used in recommendation systems.
#It performs label encoding and min-max scaling on the categorical data in the input dataframe.
#for nfm input
# The first two lines create two LabelEncoder objects.
# These are used to convert categorical variables into a numerical format that can be understood by machine learning algorithms.
# The LabelEncoder() function is called twice to create two objects, head_le and tail_le.
head_le = LabelEncoder()
tail_le = LabelEncoder()

# The fit() method is called on both objects. This method calculates the necessary parameters to perform the encoding.
head_le.fit(dt_08['head'].values)
tail_le.fit(dt_08['tail'].values)

# The MinMaxScaler is imported from the preprocessing module of the sklearn library. This is used to scale the data.
mms = MinMaxScaler(feature_range=(0,1))


In [38]:

###descriptors preparation

#The drug id and sequence are read from the respective CSV files and stored in a DataFrame called fp_id.
fp_id = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/791drug_struc.csv')['drug_id']

#RThe protein id, protein id sequence, and protein sequence are read from the respective CSV files and stored in a DataFrame called df_proseq.
#The columns of this DataFrame are then renamed to 'pro_id', 'pro_ids', and 'seq'.
df_proseq = pd.read_csv('/content/drive/MyDrive/data/yamanishi_08/989proseq.csv')
df_proseq.columns = ['pro_id','pro_ids','seq']

#The pro_id, which represents the unique identifier for each protein, is extracted from the df_proseq DataFrame.
pro_id = df_proseq['pro_id']

# The drug features are read from the morganfp.txt file using the np.loadtxt() function and stored in the drug_feats variable.
drug_feats = np.loadtxt('/content/drive/MyDrive/data/yamanishi_08/morganfp.txt',delimiter=',')

#The protein features are read from the pro_ctd.txt file using the np.loadtxt() function and stored in the pro_feats variable.
pro_feats = np.loadtxt('/content/drive/MyDrive/data/yamanishi_08/pro_ctd.txt',delimiter=',')

# The protein features are then scaled using the MinMaxScaler. The scaled features are stored in the pro_feats_scaled variable.
pro_feats_scaled = mms.fit_transform(pro_feats)

#Next, PCA is applied to reduce the dimensionality of the scaled protein features to 100 components. The reduced features are stored in the pro_feats_scaled2 variable.
pro_feats_scaled2 = PCA(n_components=100).fit_transform(pro_feats_scaled)

#The reduced protein features are then scaled again using the MinMaxScaler. The scaled features are stored in the pro_feats_scaled3 variable.
pro_feats_scaled3 = mms.fit_transform(pro_feats_scaled2)

#Finally, the fp_df and prodes_df DataFrames are created by concatenating the drug id and drug features (represented by drug_feats),
#and the protein id and protein features (represented by pro_feats_scaled3), respectively.
fp_df = pd.concat([fp_id,pd.DataFrame(drug_feats)],axis=1)
prodes_df = pd.concat([pro_id,pd.DataFrame(pro_feats_scaled3)],axis=1)


In [39]:

#Function
################################################################

# If you want to test other scenarios, just change the data path.
# But it should be noted that the hypermeters in nfm need to be adjusted.
# Typiclly, the l2_reg_dnn & l2_reg_linear = 1e-5 is enough in the warm start.
# For the cold start, the l2_reg_dnn & l2_reg_linear need to be larger, like 1e-3.

data_path = '/content/drive/MyDrive/data/yamanishi_08/data_folds/warm_start_1_10'


######This function is designed to work with 10-fold cross-validation, as it assumes there are 10 different folds for the training and testing sets.
# Therefore, the input i represents the current fold. The function loads the training and testing data for this fold and returns them.
# In addition, the function merges the positive train examples and the knowledge graph into a single dataframe,
# which is used for creating the embeddings. This is why the data dataframe includes only the head, relation, and tail attributes, without the label attribute.
def load_data(i):
    # Read the train_fold csv file. The label is included.
    train = pd.read_csv(data_path+'/train_fold_'+str(i+1)+'.csv')[['head','relation','tail','label']]

    # Select only the positive examples (label == 1) from the train set.
    train_pos = train[train['label']==1]

    # Read the test_fold csv file. The label is included.
    test = pd.read_csv(data_path+'/test_fold_'+str(i+1)+'.csv')[['head','relation','tail','label']]

    # Merge the positive train examples and the knowledge graph into a single dataframe.
    data = pd.concat([train_pos,kg])[['head','relation','tail']]

    # Return the train, train_pos, test, and data dataframes.
    return train,train_pos,test,data



####This Python function, roc_auc, computes the area under the ROC curve (AUC-ROC) for a given binary classification problem.
#It takes two parameters: y (ground truth) and pred (predicted probabilities). The function returns the AUC-ROC score.

#This line defines the function roc_auc that takes two parameters: y and pred.
def roc_auc(y,pred):

  # This line calls the roc_curve function from the metrics module (part of the Scikit-Learn library) with y and pred as parameters.
  #The roc_curve function calculates the ROC curve for the given binary classification problem, returning false positive rate (FPR), true positive rate (TPR), and thresholds.
    fpr, tpr, thresholds = metrics.roc_curve(y, pred)

    #roc_auc = metrics.auc(fpr, tpr): This line calls the auc function from the metrics module with fpr and tpr as parameters.
    #The auc function calculates the area under the ROC curve, which is the AUC-ROC score.
    roc_auc = metrics.auc(fpr, tpr)

    #return roc_auc: This line returns the computed AUC-ROC score.
    return roc_auc



###This Python function, named pr_auc, takes two parameters: y and pred. The purpose of this function is to calculate the area under the Precision-Recall curve (PR-AUC).
#the function assumes that the input arrays y and pred have the same length, as they represent the ground truth and predicted labels for a binary classification problem.
#The function will raise a ValueError if the lengths of the two input arrays do not match.
def pr_auc(y, pred):

    #The first line inside the function defines the variable precision, recall, and thresholds. It calculates these values using the precision_recall_curve function from the metrics module.
    precision, recall, thresholds = metrics.precision_recall_curve(y, pred)

    #The second line of the function calculates the PR-AUC score using the auc function from the metrics module. It takes the recall values and the precision values as input parameters.
    pr_auc = metrics.auc(recall, precision)

    #Finally, the function returns the PR-AUC score as output.The output of the function is a float number, representing the PR-AUC score.
    return pr_auc



###get_scaled_embeddings that takes a pre-trained knowledge graph embedding model and three sets of triples as input parameters.
# It outputs the scaled embeddings for the subjects and objects of the training and testing triples.
#In summary, this function first obtains the embeddings for the subject and object entities of the triples.
#Then, it concatenates these embeddings and applies normalization and dimensionality reduction techniques (MinMaxScaler and PCA)
#to obtain scaled embeddings that can be used for various downstream tasks.

#Defines the function and specifies the input parameters.
def get_scaled_embeddings(model,train_triples,test_triples,get_scaled,n_components):

    # For each triple, the function extracts the subject (head) entities and gets their embeddings from the pre-trained model. It does this for both the training and testing triples.
    [train_sub_embeddings,test_sub_embeddings] = [model.get_embeddings(x['head'].values, embedding_type='entity') for x in [train_triples,test_triples]]

    #Similarly, it extracts the object (tail) entities and gets their embeddings from the pre-trained model. Again, it does this for both the training and testing triples.
    [train_obj_embeddings,test_obj_embeddings] = [model.get_embeddings(x['tail'].values, embedding_type='entity') for x in [train_triples,test_triples]]

    #The function concatenates the subject and object embeddings for each triple in the training set.
    train_feats = np.concatenate([train_sub_embeddings,train_obj_embeddings],axis=1)

    #The function also concatenates the subject and object embeddings for each triple in the testing set.
    test_feats = np.concatenate([test_sub_embeddings,test_obj_embeddings],axis=1)

    #The function applies the MinMaxScaler (mms) to the concatenated training features to obtain a set of normalized dense features.
    train_dense_features = mms.fit_transform(train_feats)

    #The function also applies the MinMaxScaler to the concatenated testing features to obtain a set of normalized dense features.
    test_dense_features = mms.transform(test_feats)

    #If the parameter get_scaled is True, the function proceeds to apply Principal Component Analysis (PCA) to the normalized dense features.
    if get_scaled:

        #The function creates a PCA instance with the specified number of components (n_components).
        pca = PCA(n_components=n_components)

        #The function applies PCA to the normalized training dense features.
        scaled_train_dense_features = pca.fit_transform(train_dense_features)

        #The function also applies PCA to the normalized testing dense features.
        scaled_pca_test_dense_features = pca.transform(test_dense_features)

    #If the parameter get_scaled is False, the function skips the PCA step and directly assigns the normalized dense features to the output variables.
    else:

        #Assigns the normalized training dense features to the output variable.
        scaled_train_dense_features = train_dense_features

        #Assigns the normalized testing dense features to the output variable.
        scaled_pca_test_dense_features = test_dense_features

    #Returns the scaled embeddings for the subjects and objects of the training and testing triples.
    return scaled_train_dense_features,scaled_pca_test_dense_features



#In summary, the function get_features takes a dataframe data, two dataframes fp_df and prodes_df, and a boolean variable use_pro.
#It extracts drug features from fp_df and protein features from prodes_df using the head and tail columns of data.
#Then, it concatenates the features horizontally based on the value of use_pro. The resulting feature matrix is returned as the output of the function.
#This line defines a function named get_features. It takes four parameters: data, fp_df, prodes_df, and use_pro.
def get_features(data,fp_df,prodes_df,use_pro):

    #This line performs a left join of data and fp_df on the 'head' column of data and the 'drug_id' column of fp_df.
    #The left join is done because we want to keep all records from the left table (i.e., data) and the matched records from the right table (i.e., fp_df).
    #Then, the code selects the 1025 columns (from the 5th to the 1029th column) of the resulting dataframe and converts it into a numpy array using the values attribute.
    #The result is stored in the variable drug_features.
    drug_features = pd.merge(data,fp_df,how='left',left_on='head',right_on='drug_id').iloc[:,4:1029].valuesپ

    #This line performs a similar operation as the previous one, but this time it joins data and prodes_df on the 'tail' column of data and the 'pro_id' column of prodes_df.
    #Again, the code selects the 101 columns (from the 5th to the 105th column) of the resulting dataframe and converts it into a numpy array using the values attribute.
    #The result is stored in the variable pro_features.
    pro_features = pd.merge(data,prodes_df,how='left',left_on='tail',right_on='pro_id').iloc[:,4:105].values

    #This line checks the value of the variable use_pro. If it is True, it concatenates drug_features and pro_features horizontally using the np.concatenate function.
    #If use_pro is False, it directly assigns drug_features to the variable feature.
    if use_pro:
        feature = np.concatenate([drug_features,pro_features],axis=1)
    else:
        feature = drug_features

    #This line returns the final feature matrix.
    return feature


In [40]:
#'DenseFeat("des",train_des.shape[1]),'des':train_des,' is used for nfm training
#In this code, the input to get_nfm_input is the raw relational entities and their features/descriptions.
#The output is the preprocessed input data that can be used as input for a Neural Factorization Machine (NFM) model.


##Define a function named get_nfm_input with the following parameters:
# re_train_all: a dataframe of training relational entities.
# re_test_all: a dataframe of testing relational entities.
# train_feats: the training features matrix.
# test_feats: the testing features matrix.
# train_des: the training description vectors.
# test_des: the testing description vectors.
# embedding_dim: the dimensionality of the embedding space.
# pca_components: the number of PCA components to retain.
def get_nfm_input(re_train_all,re_test_all,train_feats,test_feats,train_des,test_des,embedding_dim,pca_components):

    #The next line concatenates the train_feats and train_des matrices.
    train_all_feats = np.concatenate([train_feats,train_des],axis=1)

    #Similarly, the next line concatenates the test_feats and test_des matrices.
    test_all_feats = np.concatenate([test_feats,test_des],axis=1)

    #The next line applies MinMaxScaler to the concatenated training all features matrix.
    train_all_feats_scaled = mms.fit_transform(train_all_feats)

    #The next line applies MinMaxScaler to the concatenated testing all features matrix.
    test_all_feats_scaled = mms.transform(test_all_feats)

    #The next line creates a list of feature columns, where each feature column is
    #defined as either a sparse feature (with its own unique vocabulary size and embedding dimension) or a dense feature (with a specific number of features).
    feature_columns = [SparseFeat('head',re_train_all['head'].unique().shape[0],embedding_dim=embedding_dim),
                        SparseFeat('tail',re_train_all['tail'].unique().shape[0],embedding_dim=embedding_dim),
                        DenseFeat("feats",train_all_feats_scaled.shape[1]),
                        #DenseFeat("des",train_des.shape[1])
                        ]

    #The next line creates a dictionary for the training model input.
    train_model_input = {'head':head_le.transform(re_train_all['head'].values),
                    'tail':tail_le.transform(re_train_all['tail'].values),
                     'feats':train_all_feats_scaled,
                     #'des':train_des
                    }

    #The next line creates a dictionary for the testing model input
    test_model_input = {'head':head_le.transform(re_test_all['head'].values),
                    'tail':tail_le.transform(re_test_all['tail'].values),
                    'feats':test_all_feats_scaled,
                    # 'des':test_des
                    }

    #Finally, the function returns the feature_columns, train_model_input, and test_model_input.
    return feature_columns,train_model_input,test_model_input


In [41]:

#the hypermeters(l2_reg_dnn & l2_reg_linear) need to be adjusted in cold start scenarios, like 1e-3
#The following code is used to train a neural factorization machine (NFM) model for a binary classification problem.

#The function train_nfm takes in 8 parameters: feature_columns, train_model_input, train_label, test_model_input, y, and patience.
#The output of this function is a tuple of 3 elements: roc_nfm, pr_nfm, and pred_y[:,0].
def train_nfm(feature_columns,train_model_input,train_label,test_model_input,y,patience):
   #The function initializes a neural factorization machine (NFM) model by calling the NFM class with the following parameters:
   # feature_columns, feature_columns, task='binary', dnn_hidden_units=(128,128), l2_reg_dnn=1e-5, l2_reg_linear=1e-5.
    re_model = NFM(feature_columns,feature_columns,task='binary',dnn_hidden_units=(128,128),
                    l2_reg_dnn=1e-5,l2_reg_linear=1e-5,
                    )

    #The NFM model is then compiled using the Adam optimizer with a learning rate of 1e-3 and the binary cross-entropy loss function. The metrics to be tracked during training are precision.
    re_model.compile(Adam(1e-3), "binary_crossentropy",
                metrics=[keras.metrics.Precision(name='precision'),], )

    #The function defines an early stopping callback es using the EarlyStopping class.
    #This callback will monitor the training loss and stop training if the loss has not decreased for a specified number of epochs (patience).
    es = EarlyStopping(monitor='loss',patience=patience,min_delta=0.0001,mode='min',restore_best_weights=True)

    #The function trains the NFM model by calling the fit method with the following parameters: train_model_input, train_label, batch_size=20000, epochs=2000, verbose=2, and callbacks=[es].
    #The model is trained for a maximum of 2000 epochs or until the early stopping callback decides to stop the training.
    history = re_model.fit(train_model_input, train_label,
                        batch_size=20000, epochs=2000,
                        verbose=2,
                        callbacks=[es]
                        )

    #After training, the function evaluates the model by predicting the labels for the test data.
    pred_y = re_model.predict(test_model_input, batch_size=512)

    #The function calculates the area under the ROC curve (AUC-ROC) for the test data using the roc_auc function.
    #This metric measures the model's ability to distinguish between the positive and negative classes.
    roc_nfm = roc_auc(y,pred_y[:,0])

    #The function also calculates the area under the Precision-Recall curve (AUC-PR) for the test data using the pr_auc function.
    #This metric measures the model's ability to provide relevant results when the actual relevance is considered.
    pr_nfm = pr_auc(y,pred_y[:,0])
    print(roc_nfm)
    print(pr_nfm)

    #Finally, the function returns the ROC-AUC score, the PR-AUC score, and the predicted labels for the test data.
    return roc_nfm,pr_nfm,pred_y[:,0]


In [42]:
from ampligraph.latent_features import ScoringBasedEmbeddingModel

In [43]:
import ampligraph.latent_features.loss_functions as lfs

In [44]:
from ampligraph.latent_features.loss_functions import get as get_loss

In [45]:
from ampligraph.latent_features.regularizers import get as get_regularizer

In [59]:
#This function is used to train a neural fused model (NFM) using the train data. The model will be trained for multiple folds and then evaluated on the test data.
#The performance metrics will be computed as Area Under the ROC Curve (AUC-ROC) and Area Under the Precision-Recall Curve (AUC-PR)
#Parameters:

# i: Integer. This parameter represents the fold number for the current iteration. It is used to separate the data into train and test folds.

# test_num_neg: Integer. This parameter represents the number of negative examples used in the test data.

# train_num_neg: Integer. This parameter represents the number of negative examples used in the train data.

# embedding_dim: Integer. This parameter represents the dimensionality of the embeddings.

# n_components: Integer. This parameter represents the number of principal components used for dimensionality reduction.

# use_pro: Boolean. This parameter represents whether to use the proline as a feature in the model.

# patience: Integer. This parameter represents the number of epochs with no improvement after which the training will be stopped
def train(i,test_num_neg,train_num_neg,embedding_dim,n_components,use_pro,patience):

    #This function loads the train and test data for the current fold.
    train,train_pos,test,data = load_data(i)

    # # This function initializes the DistMult model. DistMult is a neural network based approach to perform knowledge graph embedding.
    # model = DistMult(batches_count=10000,
    #     seed=0,
    #     epochs=50,
    #     k=400,
    #     #embedding_model_params={'corrupt_sides':'o'},
    #     optimizer='adam',
    #     optimizer_params={'lr':1e-3},
    #     loss='pairwise', #pairwise
    #     regularizer='LP',
    #     regularizer_params={'p':3, 'lambda':1e-5},
    #     verbose=True)

    # #This function trains the DistMult model on the train data. It uses early stopping to prevent overfitting.
    # model.fit(data.values, early_stopping =True,early_stopping_params=
    #             {
    #                 'x_valid': train_pos[['head','relation','tail']].values,       # validation set, here we use training set for validation
    #                 'criteria':'mrr',         # Uses mrr criteria for early stopping
    #                 'burn_in': 10,              # early stopping kicks in after 10 epochs
    #                 'check_interval':2,         # validates every 2th epoch
    #                 'stop_interval':3,           # stops if 3 successive validation checks are bad.
    #                 'x_filter': dt_08.values,          # Use filter for filtering out positives
    #                 'corrupt_side':'o'         # corrupt object (but not at once)
    #             })

      # Define the model parameters
    k = 400  # embedding dimension
    eta = 10  # number of negative samples per positive sample
    epochs = 1  # number of training epochs
    batches_count = 10000  # number of batches
    optim = tf.keras.optimizers.Adam(learning_rate=1e-3)
    loss = get_loss('pairwise', {'margin': 0.5})
    regularizer = get_regularizer('LP', {'p': 2, 'lambda': 1e-5})

    # Create the DistMult model
    model = ScoringBasedEmbeddingModel(
          k=k,
          eta=eta,
          scoring_type="DistMult",
          # optimizer="Adam",
          # loss="PairwiseMargin",
          # regularizer="LP",
          # regularizer_weight=1e-5,
          seed=42,
      )


    model.compile(optimizer=optim, loss=loss, entity_relation_regularizer=regularizer)

     ###earlystpe alakie
    checkpoint = tf.keras.callbacks.EarlyStopping(
       monitor="val_loss",
       min_delta=0,
       patience=5,
       verbose=1,
       mode='max',
       restore_best_weights=True
)
  ###
    model.fit(data.values,
              batch_size=10000,
              epochs=1 ,                  # Number of training epochs
              # # validation_freq=20,           # Epochs between successive validation
              # validation_burn_in=10,       # Epoch to start validation
              # validation_data=train_pos[['head','relation','tail']].values,   # Validation data
              # validation_filter=dt_08.values,     # Filter positives from validation corruptions
              callbacks=[checkpoint],       # Early stopping callback (more from tf.keras.callbacks are supported)
              # verbose=True                  # Enable stdout messages
              )





    #save_model(model, model_name_path = './eg_model/dismult_400_warm_1_10.pkl')
    #model = restore_model(model_name_path='./eg_model/dismult_400_warm_1_10.pkl')
    #The columns list contains the column names that correspond to the 'head', 'relation', and 'tail' entities of the triples.
    columns = ['head','relation','tail']

    #The model.predict(test[columns]) function generates a score for each triple in the test dataset. This score represents the probability of the triple being correct.
    test_score = model.predict(test[columns])

    #The test_label variable stores the true labels for the triples in the test dataset.
    test_label = test['label'].values


    #kge performance evaluation
    roc = roc_auc(test_label,test_score)
    pr = pr_auc(test_label,test_score)
    print(roc)
    print(pr)

    print("I am khar")
    #nfm preparation
    re_train_all = train[columns]
    re_test_all = test[columns]
    train_label = train['label']
    print("I am gav")
    #This function generates the embeddings for the train and test data.
    train_dense_features,test_dense_features = get_scaled_embeddings(model,re_train_all,re_test_all,False,n_components)

    #This function generates the additional features (chemical features, proline features, etc.) for the train and test data.
    train_des = get_features(re_train_all,fp_df,prodes_df,use_pro)
    test_des = get_features(re_test_all,fp_df,prodes_df,use_pro)



    #This function generates the input for the NFM model, which includes the combined features from the embeddings and additional features.
    feature_columns,train_model_input,test_model_input = get_nfm_input(re_train_all,re_test_all,
                                                                    train_dense_features,test_dense_features,
                                                                    train_des,test_des,
                                                                    embedding_dim,n_components)



    # This function trains the NFM model on the train data and evaluates its performance on the test data.
    roc_nfm,pr_nfm,pred_y = train_nfm(feature_columns,train_model_input,train_label,test_model_input,test_label,patience)

    #This function returns a tuple of values:

    # roc: The Area Under the ROC Curve (AUC-ROC) for the DistMult model.

    # pr: The Area Under the Precision-Recall Curve (AUC-PR) for the DistMult model.

    # roc_nfm: The Area Under the ROC Curve (AUC-ROC) for the NFM model.

    # pr_nfm: The Area Under the Precision-Recall Curve (AUC-PR) for the NFM model.

    # re_train_all: The combined train data used in the NFM model.

    # train_label: The ground truth labels for the train data.

    # re_test_all: The combined test data used in the NFM model.

    # test_label: The ground truth labels for the test data.

    # pred_y: The predicted labels for the test data.
    print("I am meymoon")
    return roc,pr,roc_nfm,pr_nfm,re_train_all,train_label,re_test_all,test_label,pred_y

In [60]:
#train and test
#the early stopping parameter in nfm, referring patience, need to be adjusted in cold start scenarios, like 15~20
################################################################
#In summary, this code performs a 10-fold cross-validation experiment with neural collaborative filtering.
#It computes and stores the ROC, PR, ROC_s, and PR_s values for each fold in a pandas DataFrame.
#The resulting DataFrame contains the summary statistics of the metrics.

ROC = []
PR = []
ROC_s = []
PR_s = []

#For loop iterates through the 10 runs.
for i in range(10):

  # print(i) prints the current iteration of the for loop.
    print(i)

    #train() is a function that runs the experiment for a given fold (i), given number of splits (10), given number of recommendations per user (10),
    #given number of features (50), given number of embeddings (200), given whether to use attention or not (True), and given the length of the session sequence (10).
    # It returns several metrics and arrays of user and item embeddings.
    roc,pr,roc_s,pr_s,re_train_all,train_label,re_test_all,test_label,pred_y = train(i,10,10,50,200,True,10)  #stores the return values of the train() function into variables.

    #assign the ground truth labels (train_label and test_label) to the respective users in the train and test sets.
    re_train_all['label'] = train_label
    re_test_all['label'] = test_label

    #re_test_all['pred'] = pred_y stores the predicted ratings (pred_y) for each user in the test set.
    re_test_all['pred'] = pred_y

    #ROC.append(roc), PR.append(pr), ROC_s.append(roc_s), PR_s.append(pr_s) append the ROC, PR, ROC_s, and PR_s values to their respective lists for each fold.
    ROC.append(roc)
    PR.append(pr)
    ROC_s.append(roc_s)
    PR_s.append(pr_s)

#creates an empty pandas DataFrame.
stable_metrics = pd.DataFrame()

# store the ROC, PR, ROC_s, and PR_s values in the respective columns of the DataFrame.
stable_metrics['roc'] = ROC
stable_metrics['pr'] = PR
stable_metrics['roc_s'] = ROC_s
stable_metrics['pr_s'] = PR_s

#prints the summary statistics of the metrics.
stable_metrics.describe()


0
12/12 [==============================] - 18s 2s/step - loss: 45584.7344
0.5083585976368159
0.09664092372773175
I am khar
I am gav


ValueError: Invalid entity type: entity